<img src="https://s3.amazonaws.com/datascienceheroes.com/EDV/360_banner_python.png" width="400px">


# Escuela de Datos Vivos 

## LAB P.D.3.4: Preparación de datos (el comienzo!)

Creado por Pablo Casas | https://escueladedatosvivos.ai   

In [1]:
import pandas as pd
from qgrid import show_grid
import seaborn as sns
from pandas_profiling import ProfileReport
from funpymodeling.exploratory import freq_tbl, status, profiling_num, cat_vars, num_vars

### Temas que vimos


- Tipos de datos para ML
- Remplazo de nulos (num y cat)
- Discretización de var

## Caso de estudio

**Datos: Encuesta permanente de hogares**

Objetivo concreto: Construir un modelo que haga la predicción de personas con ingresos mayor a 15k

Objetivo técnico: Construir un clasificador binario con random forest


Para ello -> Preparación de datos


### 1) Leemos los datos de la EPH

In [14]:
data=pd.read_csv("data/eph2.txt", sep = ",")

In [3]:
show_grid(data)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### 2) AED y detección de problemas

In [4]:
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,edad,134,0.038111,0,0.0,70,float64
1,sexo,0,0.000000,0,0.0,2,object
2,alfabeto,0,0.000000,0,0.0,2,object
3,sistema_salud,0,0.000000,0,0.0,7,object
4,nivel_educativo,66,0.018771,0,0.0,7,object
5,ocupacion_jerarquia,0,0.000000,0,0.0,4,object
6,estado_civil,0,0.000000,0,0.0,5,int64
7,ingreso_15k,0,0.000000,0,0.0,2,object


### Detectamos tres problemas: 

a) Tipo de dato incorrecto: estado_civil (int64)

b) nan en categórica: nivel_educativo

c) nan en numérica: edad

## 3) Soluciones de preparación de datos

### 3.a) ¿Cómo convertir un tipo de dato num a cat?

Tipo de dato incorrecto: estado_civil (int64) [solución]


Convertimos solo una variable:

In [15]:
# Hacemos una copia
data_original=data.copy()

In [21]:
data['estado_civil_cat']=data['estado_civil'].astype(str)
data['estado_civil_cat']

0       3
1       1
2       4
3       2
4       1
       ..
3511    1
3512    4
3513    3
3514    1
3515    3
Name: estado_civil_cat, Length: 3516, dtype: object

In [22]:
data['estado_civil_cat']=data['estado_civil'].astype('category')

In [23]:
data['estado_civil_cat']

0       3
1       1
2       4
3       2
4       1
       ..
3511    1
3512    4
3513    3
3514    1
3515    3
Name: estado_civil_cat, Length: 3516, dtype: category
Categories (5, int64): [1, 2, 3, 4, 5]

In [ ]:
# chequeo:
status(data['estado_civil_cat'])

In [8]:
data=data.drop('estado_civil', axis=1)

In [9]:
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,edad,134,0.038111,0,0.0,70,float64
1,sexo,0,0.000000,0,0.0,2,object
2,alfabeto,0,0.000000,0,0.0,2,object
3,sistema_salud,0,0.000000,0,0.0,7,object
4,nivel_educativo,66,0.018771,0,0.0,7,object
5,ocupacion_jerarquia,0,0.000000,0,0.0,4,object
6,ingreso_15k,0,0.000000,0,0.0,2,object
7,estado_civil_cat,0,0.000000,0,0.0,5,object


### 3.b) ¿Como crear la categoría 'Nulo'?

Hay `nan` en categórica: nivel_educativo [solución]


In [ ]:
status(data['nivel_educativo'])

In [ ]:
data['nivel_educativo']=data['nivel_educativo'].fillna(value="nulo") 

In [ ]:
status(data['nivel_educativo'])
freq_tbl(data['nivel_educativo'])

### 3.c) ¿Cómo tratar los nulos en variables numéricas?


Problema: Hay `nan` en numérica: edad.

**Distintos enfoques**: El de ahora es discretizar (cat) y agregar el valor "nulo" (cómo antes)

### Discretización de edad

In [ ]:
data['edad_cat'], saved_bins = pd.qcut(data['edad'], q = 5, retbins = True)

In [ ]:
data['edad_cat']

In [ ]:
freq_tbl(data['edad_cat'])

Tiene 5 categorias y nulos

In [ ]:
data['edad_cat'].fillna(value="nulo")
# error...

In [ ]:
data['edad_cat']=data['edad_cat'].cat.add_categories('nulo')

In [ ]:
data['edad_cat']

Ahora si podemos remplazar los nulos

In [ ]:
data['edad_cat']=data['edad_cat'].fillna(value="nulo")

Eliminamos la variable original

In [ ]:
data=data.drop(['edad'], axis=1)

In [ ]:
status(data)

In [ ]:
freq_tbl(data.edad_cat)

## 4) Paso final: todo a numérico (one hot encoding)

In [ ]:
data_final=pd.get_dummies(data, drop_first=True)

In [ ]:
data_final

In [ ]:
status(data_final)

Guardamos los datos para usar en otro laboratorio:

In [ ]:
import pickle
with open('data/d_eph5.pickle', 'wb') as f:
    pickle.dump(data_final, f)

--- 

## Resumen!

- Cómo convertir de num a cat

- Cómo remplazar valores nan en cat

- Cómo remplazar valores nan en num (método de discretización + nueva categoría nulo)

- Finalizamos con one hot encoding de todo.



--- 

## Ejercicios!


1) Pasar el dataset `tips` de seaborn a categórico (ahora si se puede, por ejemplo, calcular las correlaciones entre todas las variables)

2) El dataset `data/diabetes.csv` tiene muchas variables con nulos. Imputar por la mediana cada variable numérica.